In [1]:
import tensorflow as tf
import cv2
import sys
import time 
import random
import numpy as np
from collections import deque
from matplotlib import pyplot as plt
import pygame
from ple import PLE

/home/cn1lab005/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [2]:
from ple.games.catcher import Catcher
#game = Catcher()
game = Catcher(width=256, height=256)
game.screen = pygame.display.set_mode(game.getScreenDims(), 0, 32)

In [3]:
GAME  = "CATCHER"
ACTIONS = 3 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 500. # timesteps to observe before training
EXPLORE = 500. # frames over which to anneal epsilon
FINAL_EPSILON =  0.05 # final value of epsilon
INITIAL_EPSILON = 1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1

In [4]:
epsilon = 1e-9
iter_routing = 2
train_freq = 10
tf.reset_default_graph()

In [5]:
def squash(vector):
    vec_squared_norm = reduce_sum(tf.square(vector), -2, keepdims=True)
    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + epsilon)
    vec_squashed = scalar_factor * vector  # element-wise
    return(vec_squashed)
def reduce_sum(input_tensor, axis=None, keepdims=False):
    return tf.reduce_sum(input_tensor, axis=axis, keepdims=keepdims)
def softmax(logits, axis=None):
    return tf.nn.softmax(logits, axis=axis)
def routing(input, b_IJ):
    # W: [1, num_caps_i, num_caps_j * len_v_j, len_u_j, 1]
    W = tf.get_variable('Weight', shape=(1, 32, 50, 5, 1), dtype=tf.float32,
                        initializer=tf.random_normal_initializer(stddev=0.01))
    biases = tf.get_variable('bias', shape=(1, 1, 10, 5, 1))
    # A better solution is using element-wise multiply, reduce_sum and reshape
    # ops instead. Matmul [a, b] x [b, c] is equal to a series ops as
    # element-wise multiply [a*c, b] * [a*c, b], reduce_sum at axis=1 and
    # reshape to [a, c]
    input = tf.tile(input, [1, 1, 50, 1, 1])
    #assert input.get_shape() == [cfg.batch_size, 1024, 160, 8, 1]

    u_hat = reduce_sum(W * input, axis=3, keepdims=True)
    u_hat = tf.reshape(u_hat, shape=[-1, 32, 10, 5, 1])
    #assert u_hat.get_shape() == [cfg.batch_size, 1024, 10, 16, 1]

    # In forward, u_hat_stopped = u_hat; in backward, no gradient passed back from u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='stop_gradient')

    # line 3,for r iterations do
    for r_iter in range(iter_routing):
        with tf.variable_scope('iter_' + str(r_iter)):
            # line 4:
            # => [batch_size, 1024, 10, 1, 1]
            c_IJ = softmax(b_IJ, axis=2)

            # At last iteration, use `u_hat` in order to receive gradients from the following graph
            if r_iter == iter_routing - 1:
                # line 5:
                # weighting u_hat with c_IJ, element-wise in the last two dims
                # => [batch_size, 1024, 10, 16, 1]
                s_J = tf.multiply(c_IJ, u_hat)
                # then sum in the second dim, resulting in [batch_size, 1, 10, 16, 1]
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                #assert s_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]

                # line 6:
                # squash using Eq.1,
                v_J = squash(s_J)
                #assert v_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]
            elif r_iter < iter_routing - 1:  # Inner iterations, do not apply backpropagation
                s_J = tf.multiply(c_IJ, u_hat_stopped)
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                v_J = squash(s_J)

                # line 7:
                # reshape & tile v_j from [batch_size ,1, 10, 16, 1] to [batch_size, 1024, 10, 16, 1]
                # then matmul in the last tow dim: [16, 1].T x [16, 1] => [1, 1], reduce mean in the
                # batch_size dim, resulting in [1, 1024, 10, 1, 1]
                v_J_tiled = tf.tile(v_J, [1, 32, 1, 1, 1])
                u_produce_v = reduce_sum(u_hat_stopped * v_J_tiled, axis=3, keepdims=True)
                #assert u_produce_v.get_shape() == [cfg.batch_size, 1024, 10, 1, 1]

                # b_IJ += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)
                b_IJ += u_produce_v
    return(v_J)

In [7]:
def createNetwork():
    s= tf.placeholder("float", [None, 28, 28, 4])
    coeff = tf.placeholder(tf.float32, shape=(None, 32, 10, 1, 1))
    ####################### New Network COnfiguration #####################    
    w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
    w1 = tf.get_variable('w1',[6, 6, 4, 16],initializer=w_initializer)
    b1 = tf.get_variable('b1',[16],initializer=b_initializer)

    l1 = tf.nn.conv2d(s, w1, strides=[1, 2, 2, 1], padding="VALID")

    conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1))
    #print("stu",conv1)
    conv1 = tf.reshape(conv1,[-1,12,12,16])
    #print(conv1)

    capsules1 = tf.contrib.layers.conv2d(conv1, 10, kernel_size=6, stride=2, padding="VALID",
                    activation_fn = tf.nn.relu,
                    weights_initializer = tf.contrib.layers.xavier_initializer(uniform=False),
                    biases_initializer=tf.constant_initializer(0))
    #print(capsules1,"jhg")

    capsules = tf.reshape(capsules1, (-1, 32, 5, 1)) #Reshape to(batch_szie, 1152, 8, 1)
    #print(capsules)
    capsules = squash(capsules)
    #print(capsules)

    input_caps2 = tf.reshape(capsules, shape=(-1, 32, 1, capsules.shape[-2].value, 1))
    #print(capsules)

    caps2 = routing(input_caps2, coeff)
    #print(caps2)

    vector_j = tf.reshape(caps2, shape=(-1, 50))
    #print(vector_j)
    q_eval = tf.contrib.layers.fully_connected(vector_j, num_outputs=ACTIONS, activation_fn=None)
    #print(q_eval)
    readout = q_eval
    return s, coeff, readout

In [8]:
def trainNetwork(s, coeff, readout, sess):
    tick = time.time()
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices = 1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)

    ## open up a game state to communicate with emulator
    # game_state = game.GameState()
    p =  PLE(game,display_screen=True)
    actions = p.getActionSet()
    p.init()
    reward = p.act(p.NOOP)
    
    # store the previous observations in replay memory
    D = deque()
    
    # get the first state by doing nothing and preprocess the image to 80x80x4
    
    action_index =  np.random.randint(0, 3)
    reward = p.act(actions[action_index])
    if(reward != 0):
        if(reward < -1):
            r_0 = -1
        else:
            r_0 = reward
        terminal = 1
        if(np.max(readout_t) > qmax):
            qmax = np.max(readout_t) 
        print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
        qmax = -10000
    else:
        terminal = 0
        r_0 = reward
            
    x_t = p.getScreenRGB()
    x_t = cv2.cvtColor(cv2.resize(x_t, (28, 28)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    sess.run(tf.global_variables_initializer())
    # saving and loading networks
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state("saved_networks")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")
    b_IJ1 = np.zeros((1, 32, 10, 1, 1)).astype(np.float32) # batch_size=1
    b_IJ2 = np.zeros((BATCH, 32, 10, 1, 1)).astype(np.float32) # batch_size=BATCH
    epsilon = INITIAL_EPSILON
    t = 0
    score = 0
    qmax = -100000
    tick = time.time()
    while True:
        # choose an action epsilon greedily
        readout_t = readout.eval(feed_dict = {s:s_t.reshape((1,28,28,4)), coeff:b_IJ1})
        
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if random.random() <= epsilon or t <= OBSERVE:
            action_index = random.randrange(ACTIONS)
            a_t[action_index] = 1
        else:
            action_index = np.argmax(readout_t)
            a_t[action_index] = 1

        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        reward = p.act(actions[action_index])
        if(reward != 0):
            if(reward < -1):
                r_t = -1
            else:
                r_t = reward
            terminal = 1
            if(np.max(readout_t) > qmax):
                qmax = np.max(readout_t) 
            print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
            qmax = -10000
        else:
            terminal = 0
            r_t = reward
        # run the selected action and observe next state and reward
        x_t1_colored = p.getScreenRGB()
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored, (28, 28)), cv2.COLOR_BGR2GRAY)
        ret, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (28, 28, 1))
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)
        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()
        
        # only train if done observing
        if t > OBSERVE and t%train_freq==0:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s:s_j1_batch, coeff:b_IJ2 })
            #readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                # if terminal only equals reward
                if minibatch[i][4]:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})
            loss = cost.eval(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})

        # update the old values
        s_t = s_t1
        t += 1
        
        # save progress every 10000 iterations
        if t % 10000 == 0:
            saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        score += reward
        if p.game_over():
            p.reset_game()
            #print("time_step:",t,"score:",score)
            score = 0

In [9]:
def playGame():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    s, coeff, readout = createNetwork()
    trainNetwork(s, coeff, readout, sess)

In [ ]:

def main():
    playGame()

if __name__ == "__main__":
    tick = time.time()
    main()
    print("Game_Ends_in Time:",int(time.time() - tick))
    print("____________ END HERE _____________")

Could not find old network weights
time_step 44 /SCORE 0 /rt -1.0 / Q_MAX 9.731150e-02
time_step 80 /SCORE -1.0 /rt -1.0 / Q_MAX 9.708415e-02
time_step 121 /SCORE -2.0 /rt -1 / Q_MAX 9.743255e-02
time_step 168 /SCORE 0.0 /rt -1.0 / Q_MAX 9.865469e-02
time_step 205 /SCORE -1.0 /rt -1.0 / Q_MAX 9.749158e-02
time_step 244 /SCORE -2.0 /rt -1 / Q_MAX 9.634360e-02
time_step 278 /SCORE 0.0 /rt 1.0 / Q_MAX 9.566980e-02
time_step 321 /SCORE 1.0 /rt 1.0 / Q_MAX 9.772233e-02
time_step 356 /SCORE 2.0 /rt 1.0 / Q_MAX 9.744957e-02
time_step 398 /SCORE 3.0 /rt -1.0 / Q_MAX 9.800023e-02
time_step 432 /SCORE 2.0 /rt -1.0 / Q_MAX 9.827113e-02
time_step 478 /SCORE 1.0 /rt -1 / Q_MAX 9.690214e-02
time_step 525 /SCORE 0.0 /rt -1.0 / Q_MAX 9.589659e-02
time_step 561 /SCORE -1.0 /rt -1.0 / Q_MAX 9.380159e-02
time_step 605 /SCORE -2.0 /rt -1 / Q_MAX 9.414417e-02
time_step 643 /SCORE 0.0 /rt -1.0 / Q_MAX 9.292899e-02
time_step 677 /SCORE -1.0 /rt -1.0 / Q_MAX 9.230815e-02
time_step 716 /SCORE -2.0 /rt -1 / Q_M

time_step 5850 /SCORE 0.0 /rt -1.0 / Q_MAX -1.009565e-02
time_step 5894 /SCORE -1.0 /rt -1 / Q_MAX -1.033521e-02
time_step 5936 /SCORE 0.0 /rt 1.0 / Q_MAX 7.492178e-02
time_step 5974 /SCORE 1.0 /rt 1.0 / Q_MAX 3.640213e-02
time_step 6008 /SCORE 2.0 /rt 1.0 / Q_MAX 7.735216e-02
time_step 6049 /SCORE 3.0 /rt -1.0 / Q_MAX -1.122673e-02
time_step 6086 /SCORE 2.0 /rt 1.0 / Q_MAX -5.790753e-03
time_step 6125 /SCORE 3.0 /rt 1.0 / Q_MAX 4.683751e-02
time_step 6171 /SCORE 4.0 /rt -1.0 / Q_MAX -1.496824e-02
time_step 6202 /SCORE 3.0 /rt 1.0 / Q_MAX 5.722685e-02
time_step 6246 /SCORE 4.0 /rt -1 / Q_MAX -1.890272e-02
time_step 6285 /SCORE 0.0 /rt 1.0 / Q_MAX 8.556814e-02
time_step 6329 /SCORE 1.0 /rt -1.0 / Q_MAX -2.078225e-02
time_step 6367 /SCORE 0.0 /rt 1.0 / Q_MAX 6.716898e-02
time_step 6403 /SCORE 1.0 /rt -1.0 / Q_MAX -2.328566e-02
time_step 6446 /SCORE 0.0 /rt 1.0 / Q_MAX 4.537396e-02
time_step 6482 /SCORE 1.0 /rt 1.0 / Q_MAX 4.541504e-02
time_step 6513 /SCORE 2.0 /rt 1.0 / Q_MAX -2.166349e-

time_step 11482 /SCORE 0.0 /rt 1.0 / Q_MAX 1.603856e-01
time_step 11528 /SCORE 1.0 /rt -1.0 / Q_MAX -6.345461e-02
time_step 11567 /SCORE 0.0 /rt -1 / Q_MAX -8.999319e-02
time_step 11601 /SCORE 0.0 /rt 1.0 / Q_MAX 1.892188e-01
time_step 11637 /SCORE 1.0 /rt -1.0 / Q_MAX -8.981807e-02
time_step 11671 /SCORE 0.0 /rt 1.0 / Q_MAX 1.947553e-01
time_step 11709 /SCORE 1.0 /rt 1.0 / Q_MAX 1.277810e-01
time_step 11750 /SCORE 2.0 /rt 1.0 / Q_MAX 1.958919e-01
time_step 11792 /SCORE 3.0 /rt -1.0 / Q_MAX -4.261985e-02
time_step 11828 /SCORE 2.0 /rt 1.0 / Q_MAX 2.114107e-01
time_step 11870 /SCORE 3.0 /rt -1 / Q_MAX -9.452300e-02
time_step 11910 /SCORE 0.0 /rt -1.0 / Q_MAX -9.183606e-02
time_step 11952 /SCORE -1.0 /rt -1.0 / Q_MAX -9.098953e-02
time_step 11989 /SCORE -2.0 /rt -1 / Q_MAX -9.506866e-02
time_step 12034 /SCORE 0.0 /rt -1.0 / Q_MAX -9.620366e-02
time_step 12073 /SCORE -1.0 /rt -1.0 / Q_MAX -9.670510e-02
time_step 12119 /SCORE -2.0 /rt -1 / Q_MAX -8.837864e-02
time_step 12154 /SCORE 0.0 /rt

time_step 16996 /SCORE 0.0 /rt -1.0 / Q_MAX -1.025048e-01
time_step 17035 /SCORE -1.0 /rt 1.0 / Q_MAX 2.814775e-01
time_step 17076 /SCORE 0.0 /rt 1.0 / Q_MAX 8.265642e-02
time_step 17109 /SCORE 1.0 /rt 1.0 / Q_MAX 3.260256e-01
time_step 17150 /SCORE 2.0 /rt 1.0 / Q_MAX 2.185629e-01
time_step 17184 /SCORE 3.0 /rt 1.0 / Q_MAX 1.832891e-01
time_step 17218 /SCORE 4.0 /rt 1.0 / Q_MAX 1.545719e-01
time_step 17255 /SCORE 5.0 /rt -1 / Q_MAX -1.431538e-01
time_step 17297 /SCORE 0.0 /rt 1.0 / Q_MAX 3.405803e-01
time_step 17330 /SCORE 1.0 /rt 1.0 / Q_MAX 3.256665e-01
time_step 17363 /SCORE 2.0 /rt 1.0 / Q_MAX 2.679954e-01
time_step 17401 /SCORE 3.0 /rt 1.0 / Q_MAX 2.185088e-01
time_step 17447 /SCORE 4.0 /rt -1.0 / Q_MAX -1.580939e-01
time_step 17483 /SCORE 3.0 /rt 1.0 / Q_MAX 3.317077e-01
time_step 17524 /SCORE 4.0 /rt 1.0 / Q_MAX 3.532623e-01
time_step 17563 /SCORE 5.0 /rt -1.0 / Q_MAX -1.523825e-01
time_step 17602 /SCORE 4.0 /rt 1.0 / Q_MAX 3.570623e-01
time_step 17648 /SCORE 5.0 /rt -1 / Q_MAX

time_step 22646 /SCORE 0.0 /rt 1.0 / Q_MAX 4.721221e-01
time_step 22687 /SCORE 1.0 /rt 1.0 / Q_MAX 3.103143e-01
time_step 22725 /SCORE 2.0 /rt 1.0 / Q_MAX 2.572706e-01
time_step 22766 /SCORE 3.0 /rt -1.0 / Q_MAX 1.255179e-01
time_step 22803 /SCORE 2.0 /rt -1 / Q_MAX -1.201931e-01
time_step 22848 /SCORE 0.0 /rt -1.0 / Q_MAX -1.385064e-01
time_step 22894 /SCORE -1.0 /rt -1.0 / Q_MAX 1.991787e-02
time_step 22938 /SCORE -2.0 /rt -1 / Q_MAX -3.605532e-02
time_step 22975 /SCORE 0.0 /rt -1.0 / Q_MAX -1.231630e-01
time_step 23009 /SCORE -1.0 /rt -1.0 / Q_MAX -1.218421e-01
time_step 23040 /SCORE -2.0 /rt 1.0 / Q_MAX 5.060603e-01
time_step 23071 /SCORE -1.0 /rt 1.0 / Q_MAX 5.311592e-01
time_step 23113 /SCORE 0.0 /rt -1 / Q_MAX 2.851324e-02
time_step 23149 /SCORE 0.0 /rt 1.0 / Q_MAX 8.330750e-02
time_step 23183 /SCORE 1.0 /rt 1.0 / Q_MAX 5.174972e-01
time_step 23214 /SCORE 2.0 /rt 1.0 / Q_MAX 4.873627e-01
time_step 23250 /SCORE 3.0 /rt -1.0 / Q_MAX 2.799669e-02
time_step 23291 /SCORE 2.0 /rt 1.0 

...

time_step 905471 /SCORE 88.0 /rt 1.0 / Q_MAX 9.525908e-01
time_step 905505 /SCORE 89.0 /rt 1.0 / Q_MAX 9.432546e-01
time_step 905543 /SCORE 90.0 /rt 1.0 / Q_MAX 1.046301e+00
time_step 905574 /SCORE 91.0 /rt 1.0 / Q_MAX 1.010787e+00
time_step 905615 /SCORE 92.0 /rt 1.0 / Q_MAX 1.006227e+00
time_step 905653 /SCORE 93.0 /rt 1.0 / Q_MAX 8.493091e-01
time_step 905694 /SCORE 94.0 /rt 1.0 / Q_MAX 9.968299e-01
time_step 905730 /SCORE 95.0 /rt 1.0 / Q_MAX 1.013173e+00
time_step 905763 /SCORE 96.0 /rt 1.0 / Q_MAX 1.006975e+00
time_step 905801 /SCORE 97.0 /rt 1.0 / Q_MAX 1.033828e+00
time_step 905842 /SCORE 98.0 /rt 1.0 / Q_MAX 9.891202e-01
time_step 905885 /SCORE 99.0 /rt 1.0 / Q_MAX 1.009225e+00
time_step 905918 /SCORE 100.0 /rt 1.0 / Q_MAX 9.866532e-01
time_step 905956 /SCORE 101.0 /rt 1.0 / Q_MAX 9.907781e-01
time_step 905993 /SCORE 102.0 /rt 1.0 / Q_MAX 9.795748e-01
time_step 906031 /SCORE 103.0 /rt 1.0 / Q_MAX 1.024851e+00
time_step 906064 /SCORE 104.0 /rt 1.0 / Q_MAX 1.023084e+00
time_step

time_step 910668 /SCORE 27.0 /rt 1.0 / Q_MAX 1.001767e+00
time_step 910707 /SCORE 28.0 /rt 1.0 / Q_MAX 1.029726e+00
time_step 910746 /SCORE 29.0 /rt 1.0 / Q_MAX 9.826181e-01
time_step 910787 /SCORE 30.0 /rt 1.0 / Q_MAX 9.882951e-01
time_step 910823 /SCORE 31.0 /rt 1.0 / Q_MAX 1.008197e+00
time_step 910860 /SCORE 32.0 /rt -1.0 / Q_MAX -4.005093e-01
time_step 910901 /SCORE 31.0 /rt 1.0 / Q_MAX 1.008553e+00
time_step 910940 /SCORE 32.0 /rt 1.0 / Q_MAX 5.752583e-01
time_step 910983 /SCORE 33.0 /rt 1.0 / Q_MAX 9.733167e-01
time_step 911019 /SCORE 34.0 /rt 1.0 / Q_MAX 9.774844e-01
time_step 911063 /SCORE 35.0 /rt 1.0 / Q_MAX 7.996374e-01
time_step 911101 /SCORE 36.0 /rt 1.0 / Q_MAX 1.038843e+00
time_step 911142 /SCORE 37.0 /rt 1.0 / Q_MAX 9.927922e-01
time_step 911180 /SCORE 38.0 /rt 1.0 / Q_MAX 9.881756e-01
time_step 911221 /SCORE 39.0 /rt 1.0 / Q_MAX 9.963572e-01
time_step 911262 /SCORE 40.0 /rt 1.0 / Q_MAX 1.023248e+00
time_step 911300 /SCORE 41.0 /rt 1.0 / Q_MAX 9.973107e-01
time_step 91

time_step 916048 /SCORE 75.0 /rt 1.0 / Q_MAX 9.488089e-01
time_step 916089 /SCORE 76.0 /rt 1.0 / Q_MAX 9.773133e-01
time_step 916132 /SCORE 77.0 /rt 1.0 / Q_MAX 9.606732e-01
time_step 916164 /SCORE 78.0 /rt 1.0 / Q_MAX 7.998319e-01
time_step 916197 /SCORE 79.0 /rt 1.0 / Q_MAX 9.975132e-01
time_step 916231 /SCORE 80.0 /rt 1.0 / Q_MAX 1.002229e+00
time_step 916267 /SCORE 81.0 /rt 1.0 / Q_MAX 9.968404e-01
time_step 916305 /SCORE 82.0 /rt 1.0 / Q_MAX 9.426608e-01
time_step 916341 /SCORE 83.0 /rt 1.0 / Q_MAX 9.729538e-01
time_step 916382 /SCORE 84.0 /rt 1.0 / Q_MAX 9.835144e-01
time_step 916420 /SCORE 85.0 /rt 1.0 / Q_MAX 9.044647e-01
time_step 916453 /SCORE 86.0 /rt 1.0 / Q_MAX 9.684386e-01
time_step 916487 /SCORE 87.0 /rt 1.0 / Q_MAX 9.789323e-01
time_step 916520 /SCORE 88.0 /rt 1.0 / Q_MAX 1.015602e+00
time_step 916553 /SCORE 89.0 /rt 1.0 / Q_MAX 9.967307e-01
time_step 916594 /SCORE 90.0 /rt 1.0 / Q_MAX 9.951962e-01
time_step 916629 /SCORE 91.0 /rt 1.0 / Q_MAX 9.837747e-01
time_step 9166

time_step 921251 /SCORE 3.0 /rt 1.0 / Q_MAX 9.904814e-01
time_step 921286 /SCORE 4.0 /rt 1.0 / Q_MAX 9.592569e-01
time_step 921329 /SCORE 5.0 /rt 1.0 / Q_MAX 9.890705e-01
time_step 921372 /SCORE 6.0 /rt 1.0 / Q_MAX 9.834805e-01
time_step 921411 /SCORE 7.0 /rt 1.0 / Q_MAX 9.978234e-01
time_step 921450 /SCORE 8.0 /rt 1.0 / Q_MAX 1.034867e+00
time_step 921486 /SCORE 9.0 /rt 1.0 / Q_MAX 9.859510e-01
time_step 921527 /SCORE 10.0 /rt 1.0 / Q_MAX 1.006621e+00
time_step 921571 /SCORE 11.0 /rt 1.0 / Q_MAX 8.683633e-01
time_step 921610 /SCORE 12.0 /rt 1.0 / Q_MAX 1.029628e+00
time_step 921643 /SCORE 13.0 /rt 1.0 / Q_MAX 9.869093e-01
time_step 921681 /SCORE 14.0 /rt 1.0 / Q_MAX 9.535906e-01
time_step 921720 /SCORE 15.0 /rt 1.0 / Q_MAX 9.741305e-01
time_step 921753 /SCORE 16.0 /rt 1.0 / Q_MAX 9.891706e-01
time_step 921794 /SCORE 17.0 /rt 1.0 / Q_MAX 1.023787e+00
time_step 921830 /SCORE 18.0 /rt 1.0 / Q_MAX 9.733860e-01
time_step 921873 /SCORE 19.0 /rt 1.0 / Q_MAX 7.401441e-01
time_step 921907 /SCO

time_step 926441 /SCORE 8.0 /rt 1.0 / Q_MAX 1.027889e+00
time_step 926474 /SCORE 9.0 /rt 1.0 / Q_MAX 1.012492e+00
time_step 926512 /SCORE 10.0 /rt 1.0 / Q_MAX 8.054473e-01
time_step 926551 /SCORE 11.0 /rt 1.0 / Q_MAX 1.008594e+00
time_step 926592 /SCORE 12.0 /rt 1.0 / Q_MAX 9.744577e-01
time_step 926635 /SCORE 13.0 /rt 1.0 / Q_MAX 1.001243e+00
time_step 926674 /SCORE 14.0 /rt 1.0 / Q_MAX 1.023589e+00
time_step 926710 /SCORE 15.0 /rt 1.0 / Q_MAX 1.009515e+00
time_step 926743 /SCORE 16.0 /rt 1.0 / Q_MAX 9.750413e-01
time_step 926774 /SCORE 17.0 /rt 1.0 / Q_MAX 1.013661e+00
time_step 926808 /SCORE 18.0 /rt 1.0 / Q_MAX 9.950867e-01
time_step 926849 /SCORE 19.0 /rt 1.0 / Q_MAX 1.018327e+00
time_step 926888 /SCORE 20.0 /rt 1.0 / Q_MAX 8.716369e-01
time_step 926927 /SCORE 21.0 /rt 1.0 / Q_MAX 1.019652e+00
time_step 926961 /SCORE 22.0 /rt 1.0 / Q_MAX 9.561214e-01
time_step 927004 /SCORE 23.0 /rt 1.0 / Q_MAX 1.019338e+00
time_step 927042 /SCORE 24.0 /rt 1.0 / Q_MAX 1.021439e+00
time_step 927076

In [ ]:
JGD

In [7]:
tf.get_default_graph()

In [8]:
s= tf.placeholder("float", [None, 28, 28, 4])
coeff = tf.placeholder(tf.float32, shape=(None, 64, 10, 1, 1))
####################### New Network COnfiguration #####################    
w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
w1 = tf.get_variable('w1',[6, 6, 4, 32],initializer=w_initializer)
b1 = tf.get_variable('b1',[32],initializer=b_initializer)

l1 = tf.nn.conv2d(s, w1, strides=[1, 2, 2, 1], padding="VALID")

conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1))

conv1 = tf.reshape(conv1,[-1,12,12,16])
print(conv1)

capsules = tf.contrib.layers.conv2d(conv1, 20, kernel_size=6, stride=2, padding="VALID",
                activation_fn = tf.nn.relu,
                weights_initializer = tf.contrib.layers.xavier_initializer(uniform=False),
                biases_initializer=tf.constant_initializer(0))
print(capsules)
capsules = tf.reshape(capsules, (-1, 64, 5, 1)) #Reshape to(batch_szie, 1152, 8, 1)
print(capsules)
capsules = squash(capsules)
print(capsules)

input_caps2 = tf.reshape(capsules, shape=(-1, 64, 1, capsules.shape[-2].value, 1))
print(capsules)

caps2 = routing(input_caps2, coeff)
print(caps2)

vector_j = tf.reshape(caps2, shape=(-1, 160))
print(vector_j)
q_eval = tf.contrib.layers.fully_connected(vector_j, num_outputs=ACTIONS, activation_fn=None)
print(q_eval)

Tensor("Reshape:0", shape=(?, 12, 12, 16), dtype=float32)
Tensor("Conv/Relu:0", shape=(?, 4, 4, 20), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 64, 5, 1), dtype=float32)
Tensor("mul:0", shape=(?, 64, 5, 1), dtype=float32)
Tensor("mul:0", shape=(?, 64, 5, 1), dtype=float32)
Tensor("iter_1/mul:0", shape=(?, 1, 10, 5, 1), dtype=float32)
Tensor("Reshape_4:0", shape=(?, 160), dtype=float32)
Tensor("fully_connected/BiasAdd:0", shape=(?, 3), dtype=float32)


In [62]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
        #print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

(6, 6, 1, 32)
4
1152
(32,)
1
32
(6, 6, 16, 20)
4
11520
(20,)
1
20
(1, 64, 50, 5, 1)
5
16000
(1, 1, 10, 5, 1)
5
50
(160, 3)
2
480
(3,)
1
3
29257


In [ ]:
s= tf.placeholder("float", [None, 28, 28, 4])
coeff = tf.placeholder(tf.float32, shape=(None, 64, 10, 1, 1))
####################### New Network COnfiguration #####################    
w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
w1 = tf.get_variable('w1',[6, 6, 4, 32],initializer=w_initializer)
b1 = tf.get_variable('b1',[32],initializer=b_initializer)

l1 = tf.nn.conv2d(s, w1, strides=[1, 2, 2, 1], padding="VALID")

conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1))
print("stu",conv1)
conv1 = tf.reshape(conv1,[-1,12,12,32])
print(conv1)

capsules1 = tf.contrib.layers.conv2d(conv1, 20, kernel_size=6, stride=2, padding="VALID",
                activation_fn = tf.nn.relu,
                weights_initializer = tf.contrib.layers.xavier_initializer(uniform=False),
                biases_initializer=tf.constant_initializer(0))
print(capsules1,"jhg")

capsules = tf.reshape(capsules1, (-1, 64, 5, 1)) #Reshape to(batch_szie, 1152, 8, 1)
print(capsules)
capsules = squash(capsules)
print(capsules)

input_caps2 = tf.reshape(capsules, shape=(-1, 64, 1, capsules.shape[-2].value, 1))
print(capsules)

caps2 = routing(input_caps2, coeff)
print(caps2)

vector_j = tf.reshape(caps2, shape=(-1, 50))
print(vector_j)
q_eval = tf.contrib.layers.fully_connected(vector_j, num_outputs=ACTIONS, activation_fn=None)
print(q_eval)

In [ ]:
x_t = np.zeros((28,28,1))
s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
b_IJ1 = np.zeros((1, 64, 10, 1, 1)).astype(np.float32) # batch_size=1
b_IJ2 = np.zeros((BATCH, 64, 10, 1, 1)).astype(np.float32) # batch_size=BATCH
print(b_IJ2.shape)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
#readout_t = q_eval.eval(feed_dict = {s:s_t.reshape((1,28,28,4)), coeff:b_IJ1})
s_j1_batch = np.zeros((BATCH,28,28,4))
readout_j1_batch =q_eval.eval(feed_dict = {s:s_j1_batch, coeff:b_IJ2 })
print(readout_j1_batch.shape)